In [1]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np


In [2]:
# Load all the datasets
train_file_path = 'sales_train_validation_afcs2024.csv'
test_validation_path = 'sales_test_validation_afcs2024.csv'
test_evaluation_path = 'sales_test_evaluation_afcs_2024.csv'
submission_file_path = 'sample_submission_afcs2024.csv'

# Load datasets
train_data = pd.read_csv(train_file_path)
test_validation_data = pd.read_csv(test_validation_path)
test_evaluation_data = pd.read_csv(test_evaluation_path)
submission_data = pd.read_csv(submission_file_path)



In [3]:
#Reshape train data into long format
train_data_long = train_data.melt(id_vars=["id"], var_name="day", value_name="sales")
train_data_long["day"] = train_data_long["day"].str.extract("(\\d+)").astype(int)
train_data_long = train_data_long.sort_values(by=["id", "day"]).reset_index(drop=True)

# repare data for Prophet for a specific product

def prepare_data_for_prophet(data, product_id):
    """
    Prepare data for Prophet model for a specific product ID.
    Args:
        data (DataFrame): Long-format data.
        product_id (str): Product ID to filter.
    Returns:
        DataFrame: Prepared data with columns 'ds' and 'y'.
    """
    product_data = data[data['id'] == product_id][['day', 'sales']]
    product_data['ds'] = pd.date_range(start='2011-01-01', periods=len(product_data), freq='D')
    product_data = product_data.rename(columns={'sales': 'y'})
    return product_data

# Train Prophet for one product, forecast future values, and calculate RMSE

def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Select one product/store pair for analysis
product_id = train_data['id'].unique()[0]
print(f"Processing product: {product_id}")

# Prepare data for the selected product
product_data = prepare_data_for_prophet(train_data_long, product_id)
test_data = test_validation_data[test_validation_data['id'] == product_id].iloc[:, 1:].values.flatten()

# Initialize and fit the Prophet model
prophet_model = Prophet()
prophet_model.fit(product_data)



Processing product: FOODS_3_001_TX_3_validation


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpddnjb85r/yhznzbm2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpddnjb85r/yv3ce6nb.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=9357', 'data', 'file=/tmp/tmpddnjb85r/yhznzbm2.json', 'init=/tmp/tmpddnjb85r/yv3ce6nb.json', 'output', 'file=/tmp/tmpddnjb85r/prophet_model7npseplj/prophet_model-20241216232638.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:26:38 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
23:26:39 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [4]:
# Make future predictions
forecast_horizon = 30  # Forecast next 30 days
future = prophet_model.make_future_dataframe(periods=forecast_horizon)
forecast = prophet_model.predict(future)


In [11]:

# Calculate RMSE for test set
y_pred = forecast['yhat'].iloc[-forecast_horizon:]
rmse = calculate_rmse(test_data, y_pred)
print(f"RMSE for {product_id}: {rmse:.4f}")


RMSE for FOODS_3_827_TX_3_validation: 1.7720


In [ ]:
# Step 4: Save results for submission
forecast = forecast[['ds', 'yhat']].tail(forecast_horizon)
forecast['id'] = product_id
forecast['day'] = range(len(product_data) + 1, len(product_data) + 1 + forecast_horizon)
forecast_results = forecast.pivot(index='id', columns='day', values='yhat').reset_index()
forecast_results.columns = ['id'] + [f'd_{i}' for i in range(1914, 1914 + forecast_horizon)]
forecast_results.to_csv('forecast_submission.csv', index=False)

print(f"Forecasting completed for product: {product_id}. RMSE: {rmse:.4f}. Results saved to 'forecast_submission.csv'.")


In [8]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np

# Load all the datasets
train_file_path = 'sales_train_validation_afcs2024.csv'
test_validation_path = 'sales_test_validation_afcs2024.csv'
test_evaluation_path = 'sales_test_evaluation_afcs_2024.csv'
submission_file_path = 'sample_submission_afcs2024.csv'

# Load datasets
train_data = pd.read_csv(train_file_path)
test_validation_data = pd.read_csv(test_validation_path)
test_evaluation_data = pd.read_csv(test_evaluation_path)
submission_data = pd.read_csv(submission_file_path)

# Step 1: Reshape train data into long format
train_data_long = train_data.melt(id_vars=["id"], var_name="day", value_name="sales")
train_data_long["day"] = train_data_long["day"].str.extract("(\\d+)").astype(int)
train_data_long = train_data_long.sort_values(by=["id", "day"]).reset_index(drop=True)

# Step 2: Prepare data for Prophet for a specific product

def prepare_data_for_prophet(data, product_id):
    """
    Prepare data for Prophet model for a specific product ID.
    Args:
        data (DataFrame): Long-format data.
        product_id (str): Product ID to filter.
    Returns:
        DataFrame: Prepared data with columns 'ds' and 'y'.
    """
    product_data = data[data['id'] == product_id][['day', 'sales']]
    product_data['ds'] = pd.date_range(start='2011-01-01', periods=len(product_data), freq='D')
    product_data = product_data.rename(columns={'sales': 'y'})
    return product_data

# Step 3: Train Prophet for all products, forecast future values, and calculate RMSE

def calculate_rmse(y_true, y_pred):
    # Align lengths if test data is shorter
    min_length = min(len(y_true), len(y_pred))
    return np.sqrt(mean_squared_error(y_true[:min_length], y_pred[:min_length]))

forecast_horizon = test_validation_data.shape[1] - 1  # Exclude 'id' column
submission_forecast = submission_data.copy()

rmse_scores = []

for product_id in train_data['id'].unique():
    print(f"Processing product: {product_id}")
    # Prepare data for the selected product
    product_data = prepare_data_for_prophet(train_data_long, product_id)
    test_data = test_validation_data[test_validation_data['id'] == product_id].iloc[:, 1:].values.flatten()

    # Initialize and fit the Prophet model
    prophet_model = Prophet()
    prophet_model.fit(product_data)

    # Make future predictions
    future = prophet_model.make_future_dataframe(periods=forecast_horizon)
    forecast = prophet_model.predict(future)

    # Calculate RMSE for test set
    y_pred = forecast['yhat'].iloc[-forecast_horizon:]
    rmse = calculate_rmse(test_data, y_pred)
    rmse_scores.append(rmse)

    # Update the submission dataframe with predicted values
    submission_forecast.loc[submission_forecast['id'] == product_id, submission_forecast.columns[1:]] = y_pred.values

# Report overall RMSE
average_rmse = np.mean(rmse_scores)
print(f"Average RMSE: {average_rmse:.4f}")

# Save updated submission file
submission_forecast.to_csv('forecast_submission.csv', index=False)

print("Forecasting completed for all products. Results saved to 'forecast_submission.csv'.")


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpddnjb85r/ehvk1pej.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpddnjb85r/m2fexwie.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=26333', 'data', 'file=/tmp/tmpddnjb85r/ehvk1pej.json', 'init=/tmp/tmpddnjb85r/m2fexwie.json', 'output', 'file=/tmp/tmpddnjb85r/prophet_model11lon3cs/prophet_model-20241216233808.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:38:08 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


Processing product: FOODS_3_001_TX_3_validation


23:38:08 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
<ipython-input-8-2671efcd6f33>:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.38550602558759717' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submission_forecast.loc[submission_forecast['id'] == product_id, submission_forecast.columns[1:]] = y_pred.values
<ipython-input-8-2671efcd6f33>:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5973894115812413' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submission_forecast.loc[submission_forecast['id'] == product_id, submission_forecast.columns[1:]] = y_pred.values
<ipython-input-8-2671efcd6f33>:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in

Processing product: FOODS_3_002_TX_3_validation


DEBUG:cmdstanpy:input tempfile: /tmp/tmpddnjb85r/ktrz6gyx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=30585', 'data', 'file=/tmp/tmpddnjb85r/tqyu05ht.json', 'init=/tmp/tmpddnjb85r/ktrz6gyx.json', 'output', 'file=/tmp/tmpddnjb85r/prophet_modeljxhd0djc/prophet_model-20241216233809.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:38:09 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
23:38:09 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


...

Processing product: FOODS_3_826_TX_3_validation


INFO:cmdstanpy:Chain [1] start processing
23:52:10 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpddnjb85r/ul3l0h96.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpddnjb85r/eva4qohx.json
DEBUG:cmdstanpy:idx 0


Processing product: FOODS_3_827_TX_3_validation


DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=70489', 'data', 'file=/tmp/tmpddnjb85r/ul3l0h96.json', 'init=/tmp/tmpddnjb85r/eva4qohx.json', 'output', 'file=/tmp/tmpddnjb85r/prophet_modelu8wjyap_/prophet_model-20241216235210.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:52:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
23:52:10 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Average RMSE: 2.0066
Forecasting completed for all products. Results saved to 'forecast_submission.csv'.


### Average RMSE 2.0666

## Enforce real-world constraints
- non-negativity
- integers (can't buy 0.5 of a product)

In [9]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np

# Load the forecasted file provided
prophet_forecast_path = 'prophet.csv'
test_evaluation_path = 'sales_test_evaluation_afcs_2024.csv'
submission_file_path = 'sample_submission_afcs2024.csv'

# Load datasets
forecast_data = pd.read_csv(prophet_forecast_path)
test_evaluation_data = pd.read_csv(test_evaluation_path)
submission_data = pd.read_csv(submission_file_path)

# Step 1: Process forecast data (round to nearest integer and enforce non-negativity)
forecast_data.iloc[:, 1:] = forecast_data.iloc[:, 1:].apply(lambda x: np.round(np.maximum(x, 0)))

# Step 2: Calculate RMSE against test evaluation set
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scores = []
for product_id in test_evaluation_data['id'].unique():
    test_data = test_evaluation_data[test_evaluation_data['id'] == product_id].iloc[:, 1:].values.flatten()
    pred_data = forecast_data[forecast_data['id'] == product_id].iloc[:, 1:].values.flatten()
    rmse = calculate_rmse(test_data, pred_data)
    rmse_scores.append(rmse)

# Report overall RMSE
overall_rmse = np.mean(rmse_scores)
print(f"Overall RMSE after processing: {overall_rmse:.4f}")

# Step 3: Finalize submission file
submission_data.iloc[:, 1:] = forecast_data.iloc[:, 1:]
submission_data.to_csv('forecast_submission_rounded.csv', index=False)

print("Processed forecast saved to 'forecast_submission.csv'.")


Overall RMSE after processing: 2.1488
Processed forecast saved to 'forecast_submission.csv'.


<ipython-input-9-2000075befd8>:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      0.0
1      1.0
2      0.0
3      0.0
4      1.0
      ... 
818    0.0
819    0.0
820    1.0
821    1.0
822    1.0
Name: F1, Length: 823, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submission_data.iloc[:, 1:] = forecast_data.iloc[:, 1:]
<ipython-input-9-2000075befd8>:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      1.0
1      1.0
2      0.0
3      0.0
4      1.0
      ... 
818    0.0
819    0.0
820    1.0
821    1.0
822    1.0
Name: F2, Length: 823, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submission_data.iloc[:, 1:] = forecast_data.iloc[:, 1:]
<ipython-input-9-2000075befd8>:34: FutureWarning: Setting an item of incompatible dt

### Average RMSE: 2.1488